# Image Classification Pipeline - Классификация изображений

Пайплайны для:
- Transfer Learning (ResNet, EfficientNet, ViT)
- Обучение CNN с нуля
- Data Augmentation
- TensorFlow/PyTorch

In [ ]:
!pip install torch torchvision timm albumentations opencv-python pandas -q

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import timm
from PIL import Image
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

## 1. Настройки

In [ ]:
# === ВАШИ ДАННЫЕ ===
TRAIN_CSV = 'train.csv'  # CSV с колонками: image_path, label
TEST_CSV = 'test.csv'
IMAGE_DIR = './images'

# === НАСТРОЙКИ МОДЕЛИ ===
MODEL_NAME = 'efficientnet_b0'  # 'resnet50', 'vit_base_patch16_224', 'mobilenetv3_large_100'
NUM_CLASSES = 10
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-3

print(f"Модель: {MODEL_NAME}")
print(f"Количество классов: {NUM_CLASSES}")

## 2. Dataset и Augmentation

In [ ]:
# Augmentation с Albumentations
train_transforms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

test_transforms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

class ImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None, is_test=False):
        self.df = df
        self.image_dir = Path(image_dir)
        self.transform = transform
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.image_dir / row['image_path']
        image = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            image = self.transform(image=image)['image']
        
        if self.is_test:
            return image
        else:
            label = row['label']
            return image, label

In [ ]:
# Загрузка данных
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

# Создание датасетов
train_dataset = ImageDataset(train_df, IMAGE_DIR, train_transforms)
test_dataset = ImageDataset(test_df, IMAGE_DIR, test_transforms, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

## 3. Модель (Transfer Learning)

In [ ]:
# Загрузка pre-trained модели с timm
model = timm.create_model(
    MODEL_NAME,
    pretrained=True,
    num_classes=NUM_CLASSES
)
model = model.to(device)

# Loss и optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

print("✓ Модель загружена!")

## 4. Обучение

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

# Обучение
for epoch in range(EPOCHS):
    loss, acc = train_epoch(model, train_loader, criterion, optimizer, device)
    scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {loss:.4f}, Acc: {acc:.2f}%")

# Сохранение
torch.save(model.state_dict(), 'image_classifier.pth')
print("✓ Модель обучена и сохранена!")

## 5. Предсказания

In [ ]:
model.eval()
predictions = []

with torch.no_grad():
    for images in tqdm(test_loader, desc="Predicting"):
        images = images.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        predictions.extend(preds.cpu().numpy())

# Submission
submission = pd.DataFrame({
    'id': test_df.index if 'id' not in test_df.columns else test_df['id'],
    'prediction': predictions
})
submission.to_csv('image_classification_submission.csv', index=False)
print("✓ Submission сохранен!")